In [1]:
from keras.applications import vgg16

Using TensorFlow backend.


In [2]:
model = vgg16.VGG16(weights='imagenet', include_top = False ,input_shape = (224, 224, 3))

In [3]:
model.layers[0].input


<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [4]:
for layer in model.layers:
    layer.trainable = False

In [5]:
from keras.models import Sequential

In [6]:
from keras.layers import Dense
from keras.layers import Flatten

In [7]:
top_model = model.output

In [8]:
top_model = Dense(1024,activation = 'relu')(top_model)

In [9]:
top_model = Dense(512, activation = 'relu')(top_model)

In [10]:
top_model = Flatten()(top_model)

In [11]:
top_model = Dense(2, activation = 'softmax')(top_model)

In [12]:
from keras.models import Model

In [13]:
model.input

<tf.Tensor 'input_1:0' shape=(None, 224, 224, 3) dtype=float32>

In [14]:
new = Model(inputs = model.input, output = top_model)

C:\Users\tjain\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [15]:
new.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [16]:
new.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [17]:
from keras_preprocessing.image import ImageDataGenerator

In [18]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
train_set = train_datagen.flow_from_directory(
        'C:/Users/tjain/Documents/MLOps/data/train/',
        target_size=(224, 224),
        #class_mode='binary'
        batch_size=32,
        )
test_set = test_datagen.flow_from_directory(
        'C:/Users/tjain/Documents/MLOps/data/test/',
        target_size=(224, 224),
        #class_mode='binary'
        batch_size=32,
        )
new.fit(
        train_set,
        steps_per_epoch=140,
        epochs=1,
        validation_data=test_set,
        validation_steps=60)

Found 140 images belonging to 2 classes.
Found 60 images belonging to 2 classes.
Epoch 1/1
140/140 [==============================] - 1247s 9s/step - loss: 0.0485 - accuracy: 0.9829 - val_loss: 1.0642e-05 - val_accuracy: 1.0000


In [19]:
new.save('myvgg.h5')

In [20]:
scores = new.evaluate(test_set, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

2/2 [==============================] - 10s 5s/step
Test loss: 7.327193998207804e-06
Test accuracy: 1.0


In [39]:
from keras.preprocessing import image
import cv2
from PIL import Image
from numpy.random import randint
import numpy as np

In [40]:
def load_image():
    i = randint(30)
    #img = cv2.imread("data/test/tushar/tushar" +str(i)+".jpg")
    img = Image.open("data/test/anima/anima"+str(i)+".jpg" )
    #img = Image.open("data/test/tushar/tushar"+str(i)+".jpg" )
    width = 224
    height = 224
    dim = (width, height)
    # resize image
    img = img.resize(dim)
    #img = resized.reshape(1, 28, 28, 1)
    img = np.expand_dims(img, axis =0)
    
    i += 1
    return img

In [41]:
y = 'tushar'

In [42]:
new.predict(load_image())

array([[1.000000e+00, 1.813903e-17]], dtype=float32)

In [25]:
train_set.class_indices

{'anima': 0, 'tushar': 1}